## Nume studenti:
- Alexandra Manole
- Teodor Mihaescu

## Grupa: 382

# Data Set 1: Breast Cancer
### (Missing values: yes)

In [1]:
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV

In [2]:
# Citeste datele
data = pd.read_csv('./Datasets/breast-cancer.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Class       286 non-null    object
 1   Age         286 non-null    object
 2   Menopause   286 non-null    object
 3   TumorSize   286 non-null    object
 4   InvNodes    286 non-null    object
 5   NodeCaps    278 non-null    object
 6   DegMalig    286 non-null    int64 
 7   Breast      286 non-null    object
 8   BreastQuad  285 non-null    object
 9   Irradiat    286 non-null    object
dtypes: int64(1), object(9)
memory usage: 22.5+ KB


In [3]:
# Prelucreaza si scaleaza datele
def scale_category(name, max_value):
    category_splitted = np.char.split(np.array(data[name], dtype=str), sep='-')
    category_splitted = np.array([x for x in category_splitted], dtype=int)
    data[name] = np.mean(category_splitted, axis=1) / max_value

In [4]:
# Class
data['Class'] = np.where(data['Class'] == 'no-recurrence-events', 0, 1)

# Age (10-99)
scale_category('Age', 99)

# Menopause
data['Menopause'] = np.where(data['Menopause'] == 'premeno', 0.0, data['Menopause'])
data['Menopause'] = np.where(data['Menopause'] == 'lt40', 0.5, data['Menopause'])
data['Menopause'] = np.where(data['Menopause'] == 'ge40', 1.0, data['Menopause'])
data['Menopause'] = data['Menopause'].astype('float64')

# Tumor Size (0-59)
scale_category('TumorSize',59)

# Involved Nodes(0-39)
scale_category('InvNodes', 39)

# Node Caps
data['NodeCaps'] = np.where(data['NodeCaps'] == 'yes', 1, data['NodeCaps'])
data['NodeCaps'] = np.where(data['NodeCaps'] == 'no', 0, data['NodeCaps'])
# data['NodeCaps'] = data['NodeCaps'].astype('float64')

# Deg Malig
data['DegMalig'] = data['DegMalig'] / 2 - 0.5

# Breast
data['Breast'] = np.where(data['Breast'] == 'left', 0, 1)

# Breast Quad
values = np.linspace(0, 1, 5)
data['BreastQuad'] = np.where(data['BreastQuad'] == 'left_low', values[0], data['BreastQuad'])
data['BreastQuad'] = np.where(data['BreastQuad'] == 'left_up', values[1], data['BreastQuad'])
data['BreastQuad'] = np.where(data['BreastQuad'] == 'right_low', values[2], data['BreastQuad'])
data['BreastQuad'] = np.where(data['BreastQuad'] == 'right_up', values[3], data['BreastQuad'])
data['BreastQuad'] = np.where(data['BreastQuad'] == 'central', values[4], data['BreastQuad'])
# data['BreastQuad'] = data['BreastQuad'].astype('float64')

# Irradiat
data['Irradiat'] = np.where(data['Irradiat'] == 'yes', 1, 0)

# pd.set_option("display.max_rows", 20, "display.max_columns", 20)
data

,Class,Age,Menopause,TumorSize,InvNodes,NodeCaps,DegMalig,Breast,BreastQuad,Irradiat
0,0,0.348485,0.0,0.542373,0.025641,0,1.0,0,0,0
1,0,0.449495,0.0,0.372881,0.025641,0,0.5,1,0.75,0
2,0,0.449495,0.0,0.372881,0.025641,0,0.5,0,0,0
3,0,0.651515,1.0,0.288136,0.025641,0,0.5,1,0.25,0
4,0,0.449495,0.0,0.033898,0.025641,0,0.5,1,0.5,0
...,...,...,...,...,...,...,...,...,...,...
281,1,0.348485,0.0,0.542373,0.025641,0,0.5,0,0.25,0
282,1,0.348485,0.0,0.372881,0.025641,0,1.0,0,0.25,1
283,1,0.651515,1.0,0.372881,0.025641,0,0.0,1,0.25,0
284,1,0.449495,1.0,0.542373,0.102564,0,1.0,0,0,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Class       286 non-null    int32  
 1   Age         286 non-null    float64
 2   Menopause   286 non-null    float64
 3   TumorSize   286 non-null    float64
 4   InvNodes    286 non-null    float64
 5   NodeCaps    278 non-null    object 
 6   DegMalig    286 non-null    float64
 7   Breast      286 non-null    int32  
 8   BreastQuad  285 non-null    object 
 9   Irradiat    286 non-null    int32  
dtypes: float64(5), int32(3), object(2)
memory usage: 19.1+ KB


In [6]:
# Missing values imputation
"""
Observam ca avem urmatoarele missing values:
- 1 x BreastQuad
- 8 x NodeCaps

Pentru BreastQuad vom folosi un fillna dupa frecventa, ce va inlocui valoare lipsa cu cea mai itnalnita valoare, fapt care nu va influenta prea mult datele noaste. Facem acest lucru deoarece nu exista o relatie prea sigura intre localizarea cancerului mamar si celelalte atribute

Pentru NodeCaps vom folosi un IterativeImputer deoarece acest atribut are legatura cu majoritatea celorlalte atribute, acesta fiind un multivariate imputer. Desi acest lucru este mai costisitor din cauza clasificarii facute de acesta. (initial_strategy='most_frequent' deoarece avem doar yes si no, nu vrem medie, mediana sau constanta)
"""
data['BreastQuad'].fillna(data['BreastQuad'].mode()[0], inplace=True)

iterativeImputer = IterativeImputer(random_state=0, initial_strategy='most_frequent')
data_columns = data.columns
data_index = data.index
data = pd.DataFrame(iterativeImputer.fit_transform(data))
data.columns = data_columns
data.index = data_index

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Class       286 non-null    float64
 1   Age         286 non-null    float64
 2   Menopause   286 non-null    float64
 3   TumorSize   286 non-null    float64
 4   InvNodes    286 non-null    float64
 5   NodeCaps    286 non-null    float64
 6   DegMalig    286 non-null    float64
 7   Breast      286 non-null    float64
 8   BreastQuad  286 non-null    float64
 9   Irradiat    286 non-null    float64
dtypes: float64(10)
memory usage: 22.5 KB


In [8]:
# Separarea setului de date
X = data.loc[:, 'Age':]
y = data.loc[:, :'Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, shuffle=True)

# Convert to numpy arrays
X_train_mat = X_train.to_numpy()
X_test_mat = X_test.to_numpy()
y_train_col = y_train.to_numpy()
y_test_col = y_test.to_numpy()

In [9]:
# Verifica daca sunt egal distribuite
print(Counter(y_train_col[:,0]))
print(Counter(y_test_col[:,0]))

Counter({0.0: 138, 1.0: 52})
Counter({0.0: 63, 1.0: 33})


## Model 1: KNeighborsClassifier

In [10]:
# Model
"""
Principiul din spatele acestei metode este de a gasi un numar predefinit de cazuri apropiate ca distanta de noul punct care se incearca a fi prezis. Alegerea vecinilor influenteaza rezultatele modelului. Am observant ca in jurul valorii de 10 a hiperparametrului n_neighbors obtinem valori mai bune. 
Algoritmul necesita multe resurse la partea de testare deoarece parcurge datele "memorate" in intregime, pe cand cea de antrenare se intampla aproape instant ("retinerea" datelor).
"""
model1 = KNeighborsClassifier(n_neighbors=10)
model1.fit(X_train_mat, y_train_col)
y_hat = model1.predict(X_test_mat)

In [11]:
print(f'Predicted:\t{y_hat}\n')
print(f'Ground truth:\t{y_test_col}\n')
print(f'Failed:\t{sum(y_hat != y_test_col)}')

Predicted:	[0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Ground truth:	[[1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]

Failed:	[33 63 33 33 33 33 33 33 63 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33
 33 33 33 33 33 63 33 

In [12]:
print(f'Mean Absolute Error:\t{metrics.mean_absolute_error(y_test_col, y_hat)}')
print(f'Mean Squared Error:\t{metrics.mean_squared_error(y_test_col, y_hat)}')
print(f'Accuracy:\t\t{np.round(metrics.accuracy_score(y_test_col, y_hat) * 100, 3)}%')

Mean Absolute Error:	0.3333333333333333
Mean Squared Error:	0.3333333333333333
Accuracy:		66.667%


In [13]:
# Cross Validation
results_train = cross_validate(model1, X, y, scoring=['accuracy','f1'], return_train_score=True)
results_train

{'fit_time': array([0.00598502, 0.00697494, 0.00298619, 0.00199509, 0.00448513]),
 'score_time': array([0.01296544, 0.00598454, 0.00797939, 0.00897884, 0.0104003 ]),
 'test_accuracy': array([0.70689655, 0.73684211, 0.77192982, 0.75438596, 0.66666667]),
 'train_accuracy': array([0.76754386, 0.74672489, 0.74672489, 0.7510917 , 0.77292576]),
 'test_f1': array([0.        , 0.21052632, 0.38095238, 0.5       , 0.42424242]),
 'train_f1': array([0.48543689, 0.39583333, 0.34090909, 0.37362637, 0.43478261])}

In [14]:
# Cautare hiperparametrii optimi - GridSearch
parameter_grid = {'n_neighbors': list(range(1, 20)), 'p': [1, 2, 3]}
grid_search = GridSearchCV(estimator = KNeighborsClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train_mat, y_train_col)
results_gscv = cross_val_score(grid_search, X_test_mat, y_test_col, scoring='accuracy')

In [15]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.6673684210526316
{'n_neighbors': 6, 'p': 1}


## Model 2: LogisticRegression

In [16]:
# Model
"""
Aceasta metoda folosita pentru clasificare poate lucra atat cu 2 clase (regresie binomiala), cat si cu o multitudine de clase (regresie multinomiala). Practic modelul dat de aceasta regresie determina probabilitatea ca obiectul determinat de vectorul de intrare sa apartina unei clase sau celeilalte.
Acest model consta in determinarea unor ponderi corespondente datelor de intrare, astfel incat prezicerile datelor de iesire sa fie cat mai precise.

In acest caz am folosit multi_class='ovr' deoarece avem de a face cu o regresie logistica binomiala.
"""
model2 = LogisticRegression(solver='liblinear', multi_class='ovr', max_iter=1000)
model2.fit(X_train_mat, y_train_col)
y_hat = model2.predict(X_test_mat)

In [17]:
print(f'Predicted:\t{y_hat}\n')
print(f'Ground truth:\t{y_test_col}\n')
print(f'Failed:\t{sum(y_hat != y_test_col)}')

Predicted:	[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]

Ground truth:	[[1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]

Failed:	[33 33 33 33 33 33 33 33 63 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33
 33 33 33 33 33 63 63 

In [18]:
# CV Train
results_train = cross_validate(model2, X_train_mat, y_train_col, scoring=['accuracy','f1'], return_train_score=True)
results_train

{'fit_time': array([0.00199389, 0.00199246, 0.00099754, 0.00099492, 0.00099516]),
 'score_time': array([0.0029943 , 0.0019927 , 0.00300002, 0.00199437, 0.00199533]),
 'test_accuracy': array([0.81578947, 0.73684211, 0.71052632, 0.73684211, 0.71052632]),
 'train_accuracy': array([0.73684211, 0.75      , 0.75      , 0.78947368, 0.76315789]),
 'test_f1': array([0.58823529, 0.28571429, 0.26666667, 0.375     , 0.26666667]),
 'train_f1': array([0.35483871, 0.40625   , 0.36666667, 0.48387097, 0.4       ])}

In [19]:
# CV Test
results_test = cross_validate(model2, X_test_mat, y_test_col, scoring=['accuracy','f1'], return_train_score=True)
results_test

{'fit_time': array([0.0009973 , 0.0019958 , 0.00099516, 0.00099659, 0.00099921]),
 'score_time': array([0.00299263, 0.00399232, 0.0019927 , 0.00399113, 0.00199032]),
 'test_accuracy': array([0.7       , 0.68421053, 0.68421053, 0.84210526, 0.68421053]),
 'train_accuracy': array([0.73684211, 0.74025974, 0.72727273, 0.68831169, 0.75324675]),
 'test_f1': array([0.4       , 0.25      , 0.25      , 0.66666667, 0.25      ]),
 'train_f1': array([0.44444444, 0.375     , 0.4       , 0.25      , 0.45714286])}

In [20]:
# Cautare hiperparametrii optimi - GridSearch
parameter_grid = {'max_iter': [100, 1000, 10000],
    'C' : np.linspace(0.0, 2.0, 5),
    'solver': ['saga', 'liblinear', 'lbfgs']}
grid_search = GridSearchCV(estimator = LogisticRegression(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train_mat, y_train_col)
results_gscv = cross_val_score(grid_search, X_test_mat, y_test_col, scoring='accuracy')

In [21]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.7189473684210526
{'C': 1.5, 'max_iter': 100, 'solver': 'saga'}


## Model 3: DecisionTreeClassifier

In [22]:
# Model
"""
Metoda aceasta este capabila sa faca clasificarea atat binomiala, cat si multinomiala. In clasificarea aceasta nodurile testeaza valoarea unui atribut anume, muchiile corespund raspunsului testului si fac legatura cu urmatoarele noduri, iar frunzele arborelui prezic rezultatul (efectiv incadrarea in clase).
Modelul are in spate un arbore binar decizional, si ajunge la rezultate calculand asa numita "impuritate" (gini impurity), cu ajutorul careia determina cate split-uri sa faca (sau sa nu faca) la fiecare pas/nivel din structura decizionala.
"""
model3 = DecisionTreeClassifier(max_depth=2)
model3.fit(X_train_mat, y_train_col)
y_hat = model3.predict(X_test_mat)

In [23]:
print(f'Predicted:\t{y_hat}\n')
print(f'Ground truth:\t{y_test_col}\n')
print(f'Failed:\t{sum(y_hat != y_test_col)}')

Predicted:	[0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Ground truth:	[[1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]

Failed:	[33 33 33 33 33 63 33 33 63 33 33 33 33 33 33 33 33 33 33 33 33 33 33 63
 33 33 33 33 33 63 63 

In [24]:
# CV Train
results_train = cross_validate(model3, X_train_mat, y_train_col, scoring=['accuracy','f1'], return_train_score=True)
results_train

{'fit_time': array([0.00199509, 0.0010016 , 0.00099111, 0.        , 0.00098729]),
 'score_time': array([0.00299215, 0.00199032, 0.00100064, 0.00399542, 0.00299692]),
 'test_accuracy': array([0.76315789, 0.68421053, 0.76315789, 0.78947368, 0.81578947]),
 'train_accuracy': array([0.79605263, 0.76315789, 0.79605263, 0.78947368, 0.78289474]),
 'test_f1': array([0.47058824, 0.        , 0.30769231, 0.5       , 0.58823529]),
 'train_f1': array([0.49180328, 0.28      , 0.52307692, 0.48387097, 0.45901639])}

In [25]:
# CV Test
results_test = cross_validate(model3, X_test_mat, y_test_col, scoring=['accuracy','f1'], return_train_score=True)
results_test

{'fit_time': array([0.00199652, 0.        , 0.00099754, 0.        , 0.00198698]),
 'score_time': array([0.00299358, 0.00199461, 0.00099635, 0.00099611, 0.00299788]),
 'test_accuracy': array([0.6       , 0.73684211, 0.57894737, 0.84210526, 0.57894737]),
 'train_accuracy': array([0.76315789, 0.72727273, 0.75324675, 0.7012987 , 0.77922078]),
 'test_f1': array([0.2       , 0.44444444, 0.33333333, 0.72727273, 0.33333333]),
 'train_f1': array([0.52631579, 0.46153846, 0.59574468, 0.46511628, 0.62222222])}

In [67]:
# Cautare hiperparametrii optimi - GridSearch
parameter_grid = {'max_depth': list(range(3,7)),
'min_samples_split': np.linspace(1,3,5),
'min_samples_leaf': np.linspace(0.1,0.3,5),
'max_features': ['sqrt', 'log2', None],
'criterion': ['gini', 'entropy']}
grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train_mat, y_train_col)
results_gscv = cross_val_score(grid_search, X_test_mat, y_test_col, scoring='accuracy')

In [68]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.72
{'criterion': 'entropy', 'max_depth': 3, 'max_features': 'log2', 'min_samples_leaf': 0.1, 'min_samples_split': 1.0}


## Model 4: RandomForestClassifier

In [69]:
# Model
"""
Aceasta metode de clasificare consta intr-o multitudine de arbori de decizie care lucreaza ca un intreg. Din fiecare arbore rezulta o clasa predictie si clasa cu cele mai multe "voturi" devine modelul de predictie.
"""
model4 = RandomForestClassifier(max_depth=2, random_state=0, ccp_alpha=0.2)
model4.fit(X_train_mat, y_train_col)
y_hat = model4.predict(X_test_mat)

In [70]:
print(f'Predicted:\t{y_hat}\n')
print(f'Ground truth:\t{y_test_col}\n')
print(f'Failed:\t{sum(y_hat != y_test_col)}')

Predicted:	[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]



NameError: name 'y_test_colum' is not defined

In [71]:
# CV Train
results_train = cross_validate(model4, X_train_mat, y_train_col, scoring=['accuracy','f1'], return_train_score=True)
results_train

{'fit_time': array([0.28909945, 0.2031548 , 0.21346068, 0.19845247, 0.19152141]),
 'score_time': array([0.01296568, 0.01197052, 0.02390862, 0.01294875, 0.01193595]),
 'test_accuracy': array([0.68421053, 0.68421053, 0.68421053, 0.68421053, 0.65789474]),
 'train_accuracy': array([0.67763158, 0.67763158, 0.67763158, 0.67763158, 0.68421053]),
 'test_f1': array([0., 0., 0., 0., 0.]),
 'train_f1': array([0., 0., 0., 0., 0.])}

In [72]:
# CV Test
results_test = cross_validate(model4, X_test_mat, y_test_col, scoring=['accuracy','f1'], return_train_score=True)
results_test

{'fit_time': array([0.21102738, 0.18350911, 0.18949437, 0.18350673, 0.1885097 ]),
 'score_time': array([0.009974  , 0.01695991, 0.01397133, 0.0209434 , 0.01993489]),
 'test_accuracy': array([0.75      , 0.78947368, 0.73684211, 0.73684211, 0.73684211]),
 'train_accuracy': array([0.75      , 0.74025974, 0.75324675, 0.75324675, 0.75324675]),
 'test_f1': array([0., 0., 0., 0., 0.]),
 'train_f1': array([0., 0., 0., 0., 0.])}

In [73]:
# Cautare hiperparametrii optimi - GridSearch
parameter_grid = {'max_depth': list(range(3,7)),
'min_samples_split': list(range(1,4)),
'min_samples_leaf': list(range(3,7))}
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train_mat, y_train_col)
results_gscv = cross_val_score(grid_search, X_test_mat, y_test_col, scoring='accuracy')

KeyboardInterrupt: 

In [34]:
print(results_gscv.mean())
print(grid_search.best_params_)

NameError: name 'results_gscv' is not defined

## Model 5: SVC (Support Vector Classification)

In [35]:
# Model
"""

"""
model5 = SVC(C=2, gamma='auto')
model5.fit(X_train_mat, y_train_col)
y_hat = model5.predict(X_test_mat)

In [36]:
print(f'Predicted:\t{y_hat}\n')
print(f'Ground truth:\t{y_test_col}\n')
print(f'Failed:\t{sum(y_hat != y_test_col)}')

Predicted:	[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Ground truth:	[[0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]]

Failed:	[30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
 30 30 30 30 30 30 30 

In [37]:
# CV Train
results_train = cross_validate(model5, X_train_mat, y_train_col, scoring=['accuracy','f1'], return_train_score=True)
results_train

{'fit_time': array([0.00098777, 0.00299644, 0.00248933, 0.00199509, 0.00199652]),
 'score_time': array([0.00299144, 0.00598192, 0.00199103, 0.00096917, 0.00199628]),
 'test_accuracy': array([0.71052632, 0.71052632, 0.71052632, 0.71052632, 0.71052632]),
 'train_accuracy': array([0.72368421, 0.71052632, 0.71710526, 0.71052632, 0.71052632]),
 'test_f1': array([0., 0., 0., 0., 0.]),
 'train_f1': array([0.08695652, 0.        , 0.04444444, 0.        , 0.        ])}

In [38]:
# Cross Validation
results_test = cross_validate(model5, X, y, scoring=['accuracy','f1'], return_train_score=True)
results_test

{'fit_time': array([0.00498605, 0.0040226 , 0.00495982, 0.00597739, 0.00595403]),
 'score_time': array([0.00499034, 0.00199413, 0.00797105, 0.00299215, 0.00298858]),
 'test_accuracy': array([0.70689655, 0.73684211, 0.70175439, 0.61403509, 0.63157895]),
 'train_accuracy': array([0.77631579, 0.7510917 , 0.70305677, 0.74672489, 0.74235808]),
 'test_f1': array([0.        , 0.21052632, 0.        , 0.42105263, 0.48780488]),
 'train_f1': array([0.50485437, 0.31325301, 0.        , 0.45283019, 0.42718447])}

In [39]:
# Cautare hiperparametrii optimi - GridSearch
parameter_grid = {'C': np.linspace(0,1,5),
'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
'gamma': ['scale', 'auto']}
grid_search = GridSearchCV(estimator = SVC(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train_mat, y_train_col)
results_gscv = cross_val_score(grid_search, X_test_mat, y_test_col, scoring='accuracy')

NameError: name 'y_test_col' is not defined

In [40]:
print(results_gscv.mean())
print(grid_search.best_params_)

NameError: name 'results_gscv' is not defined